In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
lyric=open('/content/lorde.txt').read()
lyric[:2000]

"Well, summer slipped us underneath her tongue\nOur days and nights are perfumed with obsession\nHalf of my wardrobe is on your bedroom floor\nUse our eyes, throw our hands overboard I am your sweetheart psychopathic crush\nDrink up your movements, still I can't get enough\nI overthink your p-punctuation use\nNot my fault, just a thing that my mind do A rush at the beginning\nI get caught up, just for a minute\nBut lover, you're the one to blame, all that you're doing\nCan you hear the violence?\nMegaphone to my chest Broadcast the boom boom boom boom\nAnd make 'em all dance to it\nBroadcast the boom boom boom boom\nAnd make 'em all dance to it\nBroadcast the boom boom boom boom\nAnd make 'em all dance to it\nBroadcast the boom boom boom boom\nAnd make 'em all dance to it\nBroadcast the boom boom boom boom\nAnd make 'em all dance to it Our thing progresses, I call and you come through\nBlow all my friendships to sit in hell with you\nBut we're the greatest, they'll hang us in the Louvr

In [ ]:
corpus=lyric.lower().split('\n')
for i in range(40,60):
    print(corpus[i])

watch the wasters blow the speakers
spill my guts beneath the outdoor light
it's just another graceless night
i hate the headlines and the weather
i'm nineteen and i'm on fire
but when we're dancing i'm alright
it's just another graceless night are you lost enough?
have another drink, get lost in us
this is how we get notorious, oh
'cause i don't know
if they keep tellin' me where to go
i'll blow my brains out to the radio, oh all of the things we're taking
'cause we are young and we're ashamed
send us to perfect places
all of our heroes fading
now i can't stand to be alone
let's go to perfect places every night, i live and die
meet somebody, take 'em home
let's kiss and then take off our clothes
it's just another graceless night, 'cause all of the things we're taking


In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1
total_words

1372

In [ ]:
input_sequences = []
for line in corpus:
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)

In [ ]:
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len,
padding='pre'))
print(max_sequence_len)
print(input_sequences)

29
[[  0   0   0 ...   0 112 196]
 [  0   0   0 ... 112 196 703]
 [  0   0   0 ... 196 703  42]
 ...
 [  0   0   0 ... 207 140 140]
 [  0   0   0 ... 140 140 140]
 [  0   0   0 ... 140 140 140]]


In [ ]:
from keras.models import load_model
loaded_model = load_model('lyrics_generator.h5')

# Testing the model

In [ ]:
next_words = 50
seed_text = "i am a king"

In [ ]:
def lyrics(next_words,seed_text):
  for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-
  1, padding='pre')
    predicted = np.argmax(loaded_model.predict(token_list, verbose=0), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
      if index == predicted:
        output_word = word
        break
    seed_text += " " + output_word
  print(seed_text)

In [ ]:
lyrics(next_words,seed_text)

"i am a king in our dreams but everybody's like a fire beneath our love is a ghost lead the violence boys the radio up we had and sing and it it it it it didn't love it it it it it it alone it kissed a level beneath the fun go it it"

# Web app creation

In [ ]:
!pip install jupyter-dash

In [ ]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output


In [ ]:
df = px.data.tips()# Build App
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1('Lyrics Generator'),
    html.Div(["Next_words: ",
              dcc.Input(id='my-input', type='number')]),
    html.Div(["Seed_text: ",
              dcc.Input(id='my-input1', type='text')]),
    html.Button('Generate Lyrics', id='show-lyrics'),
    html.Div(id='body-div')
])

@app.callback(
    Output(component_id='body-div', component_property='children'),
    Input(component_id='show-lyrics', component_property='n_clicks'),
    Input(component_id='my-input', component_property='value'),
    Input(component_id='my-input1', component_property='value'),    
)
def update_output(n_clicks,input_value,input_value1):
    if n_clicks is None:
        raise PreventUpdate
    else:
        return lyrics(input_value,input_value1)
def lyrics(next_words,seed_text):
  for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-
  1, padding='pre')
    predicted = np.argmax(loaded_model.predict(token_list, verbose=0), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
      if index == predicted:
        output_word = word
        break
    seed_text += " " + output_word
  return seed_text

In [ ]:
app.run_server(mode='external', port=8060)

Dash app running on:


<IPython.core.display.Javascript object>

In [ ]:
app._terminate_server_for_port("localhost", 8060)